## 📊 Fiche Synthétique des Données

* **Source :** [Online Retail II (UCI Machine Learning Repository)](https://archive.ics.uci.edu/dataset/502/online-retail-ii)
* **Période couverte :** Du 01/12/2009 au 09/12/2011
* **Volume :** Environ 1,07 million de lignes (transactions)
* **Colonnes importantes :**
    * `Invoice`: Identifiant de la facture (essentiel pour regrouper les transactions).
    * `Quantity`: Quantité d'articles (essentiel pour le calcul du revenu).
    * `InvoiceDate`: Date de la transaction (base pour la Récence et les Cohortes).
    * `Price`: Prix unitaire (essentiel pour le calcul Monétaire).
    * `Customer ID`: Identifiant du client (base pour toute l'analyse RFM et CLV).

## Dictionnaire des Variables

Voici le détail des colonnes présentes dans le jeu de données :

| Nom de la Colonne | Type (Pandas) | Sémantique (Description) | Unités / Valeurs Exemples | |
| :--- | :--- | :--- | :--- | :--- |
| **Invoice** | `object` (string) | Identifiant unique de la facture. | `489434` |
| **StockCode** | `object` (string) | Identifiant unique du produit (SKU). | `85048` |
| **Description** | `object` (string) | Nom / description de l'article. | `PINK CHERRY LIGHTS` |
| **Quantity** | `int64` | Quantité d'articles pour cette ligne. | `12`, `48` |
| **InvoiceDate** | `datetime64[ns]` | Date et heure de la transaction. | `2009-12-01 07:45:00` |
| **Price** | `float64` | Prix unitaire de l'article. | `6.95`, `2.10` |
| **Customer ID** | `float64` | Identifiant unique du client. | `13085.0` |
| **Country** | `object` (string) | Pays de résidence du client. | `United Kingdom` |

## Qualité des données

#### Importation des librairies

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Lecture du dataset

In [ ]:
df1 = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2009-2010')
df2 = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2010-2011')

df = pd.concat([df1, df2], ignore_index=True)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [ ]:
df.head()

#### Exploration du dataset, des outliers

In [12]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,1.033036e+06,1033036,1.033036e+06,797885.000000
mean,1.007688e+01,2011-01-03 14:30:35.429549824,4.613980e+00,15313.062777
min,-8.099500e+04,2009-12-01 07:45:00,-5.359436e+04,12346.000000
25%,1.000000e+00,2010-07-05 11:38:00,1.250000e+00,13964.000000
50%,3.000000e+00,2010-12-09 13:34:00,2.100000e+00,15228.000000
75%,1.000000e+01,2011-07-27 13:17:00,4.150000e+00,16788.000000
max,8.099500e+04,2011-12-09 12:50:00,3.897000e+04,18287.000000
std,1.751976e+02,NaN,1.223975e+02,1696.466663


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1033036 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1033036 non-null  object        
 1   StockCode    1033036 non-null  object        
 2   Description  1028761 non-null  object        
 3   Quantity     1033036 non-null  int64         
 4   InvoiceDate  1033036 non-null  datetime64[ns]
 5   Price        1033036 non-null  float64       
 6   Customer ID  797885 non-null   float64       
 7   Country      1033036 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 70.9+ MB


#### Calcul et suppression des Doublons

In [56]:
df.duplicated().sum()

0

In [ ]:
df = df.drop_duplicates()

0

#### Calcul du pourcentage des valeurs manquantes par colonnes

In [9]:
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Colonnes': df.columns,
    'Nb manquants': missing_values,
    '% manquants': missing_percent
}).sort_values(by='% manquants', ascending=False)

In [10]:
missing_df

,Colonnes,Nb manquants,% manquants
Customer ID,Customer ID,235151,22.763098
Description,Description,4275,0.413829
Invoice,Invoice,0,0.000000
StockCode,StockCode,0,0.000000
Quantity,Quantity,0,0.000000
InvoiceDate,InvoiceDate,0,0.000000
Price,Price,0,0.000000
Country,Country,0,0.000000


#### 🔍 Analyse des valeurs manquantes

**Résumé des résultats**

*Customer ID :*

* Nombre de valeurs manquantes : 243 007
* Pourcentage : 22,77 %
* Impact : Critique pour les analyses RFM et CLV (impossible d’identifier le client).
* **Décision :** Exclure ces lignes pour les analyses centrées sur le client, mais les conserver pour l’analyse globale des ventes.

*Description :*

* Nombre de valeurs manquantes : 4 382
* Pourcentage : 0,41 %
* Impact : Faible, n’affecte pas les métriques financières mais peut limiter l’analyse produit.
* **Décision :** Conserver les lignes (si StockCode est présent), ou imputer par “Unknown”.

*Autres colonnes :*

Aucun NaN détecté.

**Implications pour le projet**

Les analyses RFM, cohortes et CLV doivent être basées uniquement sur les transactions avec Customer ID non nul.
Prévoir un filtre “inclure/exclure clients anonymes” dans l’application Streamlit.

In [26]:
df = df.dropna(subset=['Customer ID'])

#### InvoiceDate format date

In [18]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

#### Nombre de commandes annulées (Invoice commençant par 'C')

In [42]:
df['Invoice'].str.startswith('C').sum()

18390

#### suppression des prix = 0

In [54]:
df = df[df['Price'] > 0]

#### Changement du type (O --> str) + Suppression des tabulations avant/après 'Description'

In [58]:
df["Description"] = df["Description"].astype(str).str.strip()

C:\Users\victo\AppData\Local\Temp\ipykernel_29076\2791850656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Description"] = df["Description"].astype(str).str.strip()


#### création d'une colonne prix total (price * Quantity)

In [64]:
df['TotalPrice'] = df['Price'] * df['Quantity']

C:\Users\victo\AppData\Local\Temp\ipykernel_29076\2251169503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalPrice'] = df['Price'] * df['Quantity']


In [ ]:
df.to_csv("online_retail_clean.csv", index=False, encoding="utf-8")